3. Разработать программу (бота) для чтения CAPTCHA (Completely Automated Public Turing test to tell Computers and Humans Apart).
4. Обучить Tesseract для распознавания одной CAPTCHA. Используя свою программу, с помощью webdriver (PhantomJS или Chromedriver, gecko) заполнить форму на сайте с простой защитой от ботов.


In [2]:
from PIL import Image,ImageFilter
import pytesseract

path ='testcaptcha/1.jpg'
captcha = Image.open(path)
result = pytesseract.image_to_string(captcha)
print(result)

BPATp



In [46]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import subprocess
import requests
from PIL import Image
from PIL import ImageOps

def cleanImage(imagePath):
    image = Image.open(imagePath)
    image = image.point(lambda x: 0 if x<143 else 255)
    borderImage = ImageOps.expand(image,border=20,fill='white')
    borderImage.save(imagePath)

html = urlopen('http://www.pythonscraping.com/humans-only')
bs = BeautifulSoup(html, 'html.parser')
#Gather prepopulated form values
imageLocation = bs.find('img', {'title': 'Image CAPTCHA'})['src']
formBuildId = bs.find('input', {'name':'form_build_id'})['value']
captchaSid = bs.find('input', {'name':'captcha_sid'})['value']
captchaToken = bs.find('input', {'name':'captcha_token'})['value']

captchaUrl = 'http://pythonscraping.com'+imageLocation
urlretrieve(captchaUrl, 'captcha.jpg')
cleanImage('captcha.jpg')
p = subprocess.Popen(['tesseract', 'captcha.jpg', 'captcha'], stdout=
    subprocess.PIPE,stderr=subprocess.PIPE)
p.wait()
f = open('captcha.txt', 'r')

#Clean any whitespace characters
captchaResponse = f.read().replace(' ', '').replace('\n', '')
print('Captcha solution attempt: '+captchaResponse)

if len(captchaResponse) == 5:
    params = {'captcha_token':captchaToken, 'captcha_sid':captchaSid,   
              'form_id':'comment_node_page_form', 'form_build_id': formBuildId, 
              'captcha_response':captchaResponse, 'name':'Ryan Mitchell', 
              'subject': 'I come to seek the Grail', 
              'comment_body[und][0][value]': 
               '...and I am definitely not a bot'}
    r = requests.post('http://www.pythonscraping.com/comment/reply/10', 
                          data=params)
    responseObj = BeautifulSoup(r.text, 'html.parser')
    if responseObj.find('div', {'class':'messages'}) is not None:
        print(responseObj.find('div', {'class':'messages'}).get_text())
else:
    print('There was a problem reading the CAPTCHA correctly!')


Captcha solution attempt: PayjF
There was a problem reading the CAPTCHA correctly!


In [45]:
from PIL import Image
from scipy.ndimage.filters import gaussian_filter
import numpy
import pytesseract
from PIL import ImageFilter
# thresold1 on the first stage
th1 = 140
th2 = 140 # threshold after blurring 
sig = 1.5 # the blurring sigma


from scipy import ndimage
original = Image.open("captcha2.png")
original.save("original.png") # reading the image from the request
black_and_white =original.convert("L") #converting to black and white 
black_and_white.save("black_and_white.png")
first_threshold = black_and_white.point(lambda p: p > th1 and 255)
first_threshold.save("first_threshold.png")
blur=numpy.array(first_threshold) #create an image array
blurred = gaussian_filter(blur, sigma=sig)
blurred = Image.fromarray(blurred)
blurred.save("blurred.png")
final = blurred.point(lambda p: p > th2 and 255)
final = final.filter(ImageFilter.EDGE_ENHANCE_MORE)
final = final.filter(ImageFilter.SHARPEN)
final.save("final.png")
number = pytesseract.image_to_string(Image.open('final.png'))
print (number)

PA’Pp

